#  基于MindSpore实现LSTM算法 
本实验基于MindSpore构建LSTM模型，输出模型的预测结果和状态信息的形状。
## 1 实验目的
1.通过实验了解LSTM算法

2.基于MindSpore中实现LSTM算法
## 2 LSTM算法原理介绍
LSTM四个函数层与具体介绍如下：

(1)第一个函数层：遗忘门
    ![jupyter](./Figures/fig001.png)
 
对于上一时刻LSTM中的单元状态，一些信息可能会随着时间的流逝而过时。为了不让过多记忆影响神经网络对现在输入的处理，我们应该选择性遗忘一些在之前单元状态中的分量——这个工作就交给了遗忘门。

每一次输入一个新的输入，LSTM会先根据新的输入和上一时刻的输出决定遗忘之前的哪些记忆——输入和上一步的输出会整合为一个单独的向量，然后通过sigmoid神经层，最后点对点的乘在单元状态上。因为sigmoid 函数会将任意输入压缩到 (0,1) 的区间上，我们可以非常直觉的得出这个门的工作原理 —— 如果整合后的向量某个分量在通过sigmoid层后变为0，那么显然单元状态在对位相乘后对应的分量也会变成0，换句话说，遗忘了这个分量上的信息；如果某个分量通过sigmoid层后为1，单元状态会“保持完整记忆”。不同的sigmoid输出会带来不同信息的记忆与遗忘。通过这种方式，LSTM可以长期记忆重要信息，并且记忆可以随着输入进行动态调整。下面的公式可以用来描述遗忘门的计算，其中f_t就是sigmoid神经层的输出向量：

$f_t=σ(W_f∙[h_(t-1),x_t ]+b_f)$

（2）第二个、第三个函数层：记忆门
记忆门是用来控制是否将在t时刻（现在）的数据并入单元状态中的控制单位。首先，用tanh函数层将现在的向量中的有效信息提取出来，然后使用sigmoid函数来控制这些记忆要放多少进入单元状态。这两者结合起来就可以做到：

 ![jupyter](./Figures/fig002.png)
 
从当前输入中提取有效信息；对提取的有效信息做出筛选，为每个分量做出评级(0 ~ 1)，评级越高的最后会有越多的记忆进入单元状态。下面的公式可以分别表示这两个步骤在LSTM中的计算：

$C_{t}^{'}=tanh⁡(W_c∙[h_{(t-1)},x_t ]+b_c)$

$i_t=σ(W_i∙[h_{(t-1)},x_t ]+b_i)$

（3）第四个函数层：输出门

输出门就是LSTM单元用于计算当前时刻的输出值的神经层。输出层会先将当前输入值与上一时刻输出值整合后的向量用sigmoid函数提取其中的信息，然后，会将当前的单元状态通过tanh函数压缩映射到区间(-1, 1)中，将经过tanh函数处理后的单元状态与sigmoid函数处理后的单元状态，整合后的向量点对点的乘起来就可以得到LSTM在 t时刻的输出。

LSTM模型是由时刻的输入词$X_{t}$ ，细胞状态$C_{t}$，临时细胞状态$\widetilde{C_{t}} $，隐层状态$h_{t}$，遗忘门$f_{t}$，记忆门$i_{t}$，输出门$ o_{t}$组成。LSTM的计算过程可以概括为:通过对细胞状态中信息遗忘和记忆新的信息使得对后续时刻计算有用的信息得以传递，而无用的信息被丢弃，并在每个时间步都会输出隐层状态$h_{t}$ ，其中遗忘、记忆与输出由通过上个时刻的隐层状态$h_{t-1}$和当前输入$X_{t}$计算出来的遗忘门$f_{t}$，记忆门$ i_{t}$，输出门$o_{t}$来控制。

LSTM总体框架图如下：
 ![jupyter](./Figures/fig003.png)


## 3 实验环境
### 实验环境要求
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4 数据处理
### 4.1 数据准备
（1）训练数据

x 是一个形状为 [3, 5, 10] 的三维张量，其中3表示批量大小，5表示时间步数，10表示每个时间步的特征维度。

h0 是一个形状为 [2, 3, 16] 的三维张量，其中2表示LSTM层数（layers），3表示批量大小，16表示每个LSTM单元的隐藏状态维度。

c0是一个形状为 [2, 3, 16] 的三维张量，其中2表示LSTM层数，3表示批量大小，16表示每个LSTM单元的记忆状态维度。

这些张量是作为输入数据和初始状态传递给LSTM模型进行预测的。
其中，x 是输入序列的特征表示，h0 是初始的隐藏状态，c0 是初始的记忆状态。

（2）测试数据

test_input 是一个形状为 [2, 4, 10] 的三维张量，其中2表示批量大小，4表示时间步数，10表示每个时间步的特征维度。这个测试输入数据是通过使用 NumPy 的 np.random.randn 函数生成的，生成的随机数服从标准正态分布，然后通过 ms.Tensor 转换为 MindSpore 张量。

test_h0 是一个形状为 [2, 2, 16] 的三维张量，其中2表示LSTM层数，2表示批量大小，16表示每个LSTM单元的隐藏状态维度。这个测试初始隐藏状态数据也是通过 np.random.randn 生成的随机数，然后转换为 MindSpore 张量。

test_c0 是一个形状为 [2, 2, 16] 的三维张量，其中2表示LSTM层数，2表示批量大小，16表示每个LSTM单元的记忆状态维度。这个测试初始细胞状态数据同样是通过 np.random.randn 生成的随机数，并转换为 MindSpore 张量。

这些测试数据用于模型的测试部分，以验证模型在具有不同输入和初始状态的情况下的预测和状态输出。
### 4.2 数据加载

In [1]:
import mindspore as ms
import numpy as np
# 定义训练数据
x = ms.Tensor(np.ones([3, 5, 10]).astype(np.float32))
h0 = ms.Tensor(np.ones([1 * 2, 3, 16]).astype(np.float32))
c0 = ms.Tensor(np.ones([1 * 2, 3, 16]).astype(np.float32))
# 打印训练数据
print("Input shape:", x.shape)
print("Initial hidden state shape:", h0.shape)
print("Initial cell state shape:", c0.shape)

Input shape: (3, 5, 10)
Initial hidden state shape: (2, 3, 16)
Initial cell state shape: (2, 3, 16)


In [2]:
# 模型测试数据
test_input = ms.Tensor(np.random.randn(2, 4, 10).astype(np.float32))  
test_h0 = ms.Tensor(np.random.randn(1 * 2, 2, 16).astype(np.float32))  
test_c0 = ms.Tensor(np.random.randn(1 * 2, 2, 16).astype(np.float32)) 
# 打印测试数据
print("tset shape:", test_input.shape)
print("Initial hidden state shape:",test_h0.shape)
print("Initial cell state shape:", test_c0.shape)

tset shape: (2, 4, 10)
Initial hidden state shape: (2, 2, 16)
Initial cell state shape: (2, 2, 16)


## 5 模型构建
（1）导入Python库&模块并配置运行信息

导入依赖包
代码如下：

In [3]:
#导入MindSpore框架
import mindspore as ms
#导入numpy库
import numpy as np

（2）定义参数变量

input_size 表示输入数据的特征维度，这里设置为10；

hidden_size 表示LSTM单元的隐藏状态维度，这里设置为16；

num_layers 表示LSTM层数，即LSTM单元的堆叠层数，这里设置为2。

In [4]:
#三个参数变量
input_size=10
hidden_size=16
num_layers=2

（3）数据的读取与处理

x 是输入序列的特征表示，是一个形状为 [3, 5, 10] 的三维张量，其中3表示批量大小，5表示时间步数，10表示每个时间步的特征维度。

h0 是初始的隐藏状态，是一个形状为 [2, 3, 16] 的三维张量，其中2表示LSTM层数（layers），3表示批量大小，16表示每个LSTM单元的隐藏状态维度。

c0初始的记忆状态，是一个形状为 [2, 3, 16] 的三维张量，其中2表示LSTM层数，3表示批量大小，16表示每个LSTM单元的记忆状态维度。

In [5]:
# 定义训练数据
x = ms.Tensor(np.ones([3, 5, 10]).astype(np.float32))
h0 = ms.Tensor(np.ones([1 * 2, 3, 16]).astype(np.float32))
c0 = ms.Tensor(np.ones([1 * 2, 3, 16]).astype(np.float32))
# 打印训练数据
print("Input shape:", x.shape)
print("Initial hidden state shape:", h0.shape)
print("Initial cell state shape:", c0.shape)

Input shape: (3, 5, 10)
Initial hidden state shape: (2, 3, 16)
Initial cell state shape: (2, 3, 16)


(4)模型构建

根据处理后的数据集，定义LSTM网络，构建LSTM网络模型。
根据提示，补充“构建模型 ”部分的代码：

In [6]:
# 定义模型
class LSTMModel(ms.nn.Cell):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = ms.nn.LSTM(input_size, hidden_size, num_layers, has_bias=True, batch_first=True, bidirectional=False)

    def construct(self, x, h0, c0):
        output, (hn, cn) = self.lstm(x, (h0, c0))
        return output, hn, cn
# 构建模型
#请补充代码

In [7]:
#参考答案：
net = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)

## 6 模型训练
根据建立的LSTM模型，对模型进行训练：

In [8]:
output, hn, cn = net(x, h0, c0)
# 打印结果
print("Output shape:", output.shape)
print("Hidden state shape:", hn.shape)
print("Cell state shape:", cn.shape)

Output shape: (3, 5, 16)
Hidden state shape: (2, 3, 16)
Cell state shape: (2, 3, 16)


## 7 模型测试
根据处理后的测试数据以及建立的LSTM模型，对模型进行测试：

In [9]:
test_output, test_hn, test_cn = net(test_input, test_h0, test_c0)
print("Test Output shape:", test_output.shape)
print("Test Hidden state shape:", test_hn.shape)
print("Test Cell state shape:", test_cn.shape)

Test Output shape: (2, 4, 16)
Test Hidden state shape: (2, 2, 16)
Test Cell state shape: (2, 2, 16)


## 8 实验总结
本实验介绍了LSTM算法的原理，并按照步骤基于MindSpore实现了LSTM算法，并包含了对模型的测试部分，用于验证模型在随机输入数据上的预测结果和状态信息。